In [ ]:
import pandas as pd
import numpy as np

# 讀取合併後的原始檔
input_path = '../data/interim/steam_merged_raw.csv' 
df = pd.read_csv(input_path, low_memory=False)

print(f"資料大小: {df.shape}")
print(f"原始欄位: {df.columns}")
df['name'] = df['name_info'].fillna(df['name_metrics'])

# 定義要保留的欄位
target_columns = [
    # --- 識別與基本資訊 ---
    'steam_appid', 
    'name', 
    'developers', 
    'publishers', 
    'release_date',
    
    # --- 遊戲特徵 ---
    'required_age',         # 分析成人遊戲佔比
    'platforms',            # 平台 (windows/mac/linux)
    'metacritic',           # 專家評分
    'genres',               # 遊戲類型
    'recommendations',      # 推薦數
    'supported_languages',  # 支援語言
    'achievements',         # 成就數
    'categories',           # 類別 (如單人/多人/合作等)
    'is_free',              # 是否免費遊玩
    
    # --- 數據表現 ---
    'positive',             # 好評數
    'negative',             # 負評數
    'owners',               # 擁有者 
    'ccu',                  # 同時在線人數
    
    # --- 時間相關 ---
    'average_forever',      # 平均遊玩時間
    'median_forever',
    'average_2weeks',
    'median_2weeks',
    
    # --- 價格相關 ---
    'price',                # 價格 
    'initialprice',         # 初始價格
    'discount'              # 折扣
]

# 執行篩選
df_clean = df[target_columns].copy()
display(df_clean.head(5))

資料大小: (82413, 56)
原始欄位: Index(['Unnamed: 0', 'steam_appid', 'name_metrics', 'developer', 'publisher',
       'score_rank', 'positive', 'negative', 'userscore', 'owners',
       'average_forever', 'average_2weeks', 'median_forever', 'median_2weeks',
       'price', 'initialprice', 'discount', 'ccu', 'type', 'name_info',
       'required_age', 'is_free', 'controller_support', 'dlc',
       'detailed_description', 'about_the_game', 'short_description',
       'fullgame', 'supported_languages', 'header_image', 'website',
       'pc_requirements', 'mac_requirements', 'linux_requirements',
       'legal_notice', 'drm_notice', 'ext_user_account_notice', 'developers',
       'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
       'platforms', 'metacritic', 'reviews', 'categories', 'genres',
       'screenshots', 'movies', 'recommendations', 'achievements',
       'release_date', 'support_info', 'background', 'content_descriptors'],
      dtype='object')


,steam_appid,name,developers,publishers,release_date,required_age,platforms,metacritic,genres,recommendations,...,negative,owners,ccu,average_forever,median_forever,average_2weeks,median_2weeks,price,initialprice,discount
0,10,Counter-Strike,['Valve'],['Valve'],"{'coming_soon': False, 'date': '1 Nov, 2000'}",0.0,"{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...","[{'id': '1', 'description': 'Action'}]",{'total': 162153},...,6427,"10,000,000 .. 20,000,000",7323,12222,204,563,88,199.0,999.0,80.0
1,20,Team Fortress Classic,['Valve'],['Valve'],"{'coming_soon': False, 'date': '1 Apr, 1999'}",0.0,"{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]",{'total': 6647},...,1136,"1,000,000 .. 2,000,000",66,361,15,6722,6722,499.0,499.0,0.0
2,30,Day of Defeat,['Valve'],['Valve'],"{'coming_soon': False, 'date': '1 May, 2003'}",0.0,"{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...","[{'id': '1', 'description': 'Action'}]",{'total': 4318},...,688,"5,000,000 .. 10,000,000",87,859,23,3485,3604,499.0,499.0,0.0
3,40,Deathmatch Classic,['Valve'],['Valve'],"{'coming_soon': False, 'date': '1 Jun, 2001'}",0.0,"{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]",{'total': 2340},...,545,"5,000,000 .. 10,000,000",7,353,10,4,4,499.0,499.0,0.0
4,50,Half-Life: Opposing Force,['Gearbox Software'],['Valve'],"{'coming_soon': False, 'date': '1 Nov, 1999'}",0.0,"{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': '1', 'description': 'Action'}]",{'total': 22873},...,1198,"2,000,000 .. 5,000,000",74,528,162,78,78,499.0,499.0,0.0


In [ ]:
import pandas as pd
import numpy as np
import ast 

# --- 處理 (Developers, Publishers) ---
def parse_list_string(text):
    """把 "['Valve', 'Hidden Path']" 變成 "Valve, Hidden Path" """
    if pd.isna(text): return ""
    try:
        items = ast.literal_eval(str(text))
        if isinstance(items, list):
            return ", ".join(items)
        return str(text)
    except:
        return str(text)

# --- 處理 {'key': val}  ---
def parse_dict_field(text, key_name):
    """
    從字典字串中提取指定的 key。
    例如從 "{'total': 100}" 提取 'total' -> 100
    """
    if pd.isna(text) or text == '[]': return None
    try:
        # 把字串變成字典
        data = ast.literal_eval(text)
        
        if isinstance(data, list):
            if len(data) > 0 and isinstance(data[0], dict):
                return data[0].get(key_name)
            return None
            
        # 如果是字典
        if isinstance(data, dict):
            return data.get(key_name)
            
        return None
    except:
        return None

# --- 處理日期 ---
def clean_release_date(date_str):
    """ 從 {'date': '1 Nov, 2000', ...} 抓出日期 """
    val = parse_dict_field(date_str, 'date')
    if val:
        return val
    return date_str # 解析失敗，回傳原本

# --- 處理價格 (Cents to Dollars) ---
def clean_price(price_val):
    if pd.isna(price_val): return 0.0
    
    val_str = str(price_val).lower()
    if 'free' in val_str or 'demo' in val_str:
        return 0.0
    
    try:
        # cents 轉換成 dollars
        return float(val_str) / 100.0
    except:
        return 0.0

# --- 處理類別/標籤 (Genres, Categories) ---
def parse_genres_and_categories(text):
    """ 把 [{'description': 'Action'}, ...] 變成 'Action' """
    if pd.isna(text): return ""
    try:
        data = ast.literal_eval(text)
        if isinstance(data, list):
            # 取出所有 description 並串起來
            desc_list = [item.get('description', '') for item in data if isinstance(item, dict)]
            return ", ".join(filter(None, desc_list)) # filter(None) 去除空字串
        return ""
    except:
        return ""
    
# --- 處理銷量 (Owners) ---
def clean_owners_simple(x):
    if pd.isna(x): return 0
    s = str(x).replace(',', '').replace('+', '')
    parts = s.split(' .. ')
    try:
        if len(parts) == 2: return (int(parts[0]) + int(parts[1])) / 2
        return float(parts[0])
    except: return 0

In [ ]:
# 解決第 66, 67, 70 列 (錯位問題)
# 只保留 release_date 是以 '{' 開頭的資料
valid_rows = df_clean['release_date'].astype(str).str.strip().str.startswith('{')
df_clean = df_clean[valid_rows]
print(f"剔除錯位資料後剩餘: {len(df_clean)}")

# 解決第 27 列 (垃圾資料)
# 剔除名字包含 "TestApp" 的
df_clean = df_clean[~df_clean['name'].astype(str).str.contains('TestApp', case=False)]
df_clean = df_clean[pd.to_numeric(df_clean['steam_appid'], errors='coerce').notnull()]

df_processed = df_clean.copy()

print("清洗 Developers & Publishers")
df_processed['developers'] = df_processed['developers'].apply(parse_list_string)
df_processed['publishers'] = df_processed['publishers'].apply(parse_list_string)

print("解析複雜欄位 (Metacritic, Recommendations, Genres)")
# Metacritic 取 'score'
df_processed['metacritic_score'] = df_processed['metacritic'].apply(lambda x: parse_dict_field(x, 'score'))

# Recommendations 取 'total'
df_processed['recommendations_count'] = df_processed['recommendations'].apply(lambda x: parse_dict_field(x, 'total'))

# Genres 和 Categories (取 description)
df_processed['genres'] = df_processed['genres'].apply(parse_genres_and_categories)
df_processed['categories'] = df_processed['categories'].apply(parse_genres_and_categories)

# Achievements ({'total': 147, ...})
df_processed['achievements_count'] = df_processed['achievements'].apply(lambda x: parse_dict_field(x, 'total'))
df_processed['achievements_count'] = df_processed['achievements_count'].fillna(0) # 填補空值

print("處理日期")
# 先從字典取出字串，再轉 datetime
df_processed['release_date_clean'] = df_processed['release_date'].apply(clean_release_date)
df_processed['release_date_parsed'] = pd.to_datetime(df_processed['release_date_clean'], errors='coerce')
df_processed['release_year'] = df_processed['release_date_parsed'].dt.year

print("處理價格")
# 處理價格 (Cents to Dollars)
df_processed['price_clean'] = df_processed['price'].apply(clean_price)

print("處理銷量 (Owners)")
df_processed['owners_avg'] = df_processed['owners'].apply(clean_owners_simple)

# 建立好評率
df_processed['total_reviews'] = df_processed['positive'] + df_processed['negative']
df_processed['review_score'] = np.where(df_processed['total_reviews'] > 0, 
                                        df_processed['positive'] / df_processed['total_reviews'], 0)
print(df_processed.shape)


剔除錯位資料後剩餘: 82201
清洗 Developers & Publishers
解析複雜欄位 (Metacritic, Recommendations, Genres)
處理日期
處理價格
處理銷量 (Owners)
(82201, 35)


In [79]:
# 檢查欄位
check_cols = [
    'name', 
    'developers',          
    'metacritic_score', 
    'recommendations_count',
    'genres',              
    'categories',          
    'achievements_count',
    'release_year',   
    'price_clean',
    'owners_avg',
    'review_score',
]

display(df_processed[check_cols].head(5))
print(df_processed.shape)

,name,developers,metacritic_score,recommendations_count,genres,categories,achievements_count,release_year,price_clean,owners_avg,review_score
0,Counter-Strike,Valve,88.0,162153.0,Action,"Multi-player, PvP, Online PvP, Shared/Split Sc...",0.0,2000.0,1.99,15000000.0,0.974317
1,Team Fortress Classic,Valve,NaN,6647.0,Action,"Multi-player, PvP, Online PvP, Shared/Split Sc...",0.0,1999.0,4.99,1500000.0,0.869993
2,Day of Defeat,Valve,79.0,4318.0,Action,"Multi-player, Camera Comfort, Color Alternativ...",0.0,2003.0,4.99,7500000.0,0.903126
3,Deathmatch Classic,Valve,NaN,2340.0,Action,"Multi-player, PvP, Online PvP, Shared/Split Sc...",0.0,2001.0,4.99,7500000.0,0.827695
4,Half-Life: Opposing Force,Gearbox Software,NaN,22873.0,Action,"Single-player, Multi-player, Custom Volume Con...",0.0,1999.0,4.99,3500000.0,0.953132


(82201, 35)


In [80]:
import os
os.makedirs('../data/processed', exist_ok=True) 

save_path = '../data/processed/steam_cleaned.csv'
df_processed.to_csv(save_path, index=False, encoding='utf-8-sig') # 使用 utf-8-sig 讓 Excel 不會有亂碼